In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
from keras import backend as K
from keras.models import Sequential, Model, load_model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Lambda, Permute, Dropout, add, dot, concatenate, multiply
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re
import tensorflow as tf
from nltk.tokenize import word_tokenize
from keras.callbacks import ModelCheckpoint
import datetime
from itertools import chain
from process_data import load_entities, save_pickle, load_pickle, load_kv_pairs, lower_list, vectorize, vectorize_kv, find_ngrams

Using TensorFlow backend.


In [4]:
is_babi = False
if is_babi:
    train_data = load_task('./data/tasks_1-20_v1-2/en/qa5_three-arg-relations_train.txt', is_babi)
    test_data = load_task('./data/tasks_1-20_v1-2/en/qa5_three-arg-relations_test.txt', is_babi)
else:
    # mem_maxlen         = 100 # 1つのエピソードに関連しているKVの数に対する制限
    train_data         = load_pickle('pickle/mov_task1_qa_pipe_train.pickle')
    test_data          = load_pickle('pickle/mov_task1_qa_pipe_test.pickle')
    kv_pairs           = load_pickle('pickle/mov_kv_pairs.pickle')
    train_k            = np.array(load_pickle('pickle/mov_train_k.pickle'))
    train_v            = np.array(load_pickle('pickle/mov_train_v.pickle'))
    test_k             = np.array(load_pickle('pickle/mov_test_k.pickle'))
    test_v             = np.array(load_pickle('pickle/mov_test_v.pickle'))
    entities           = load_pickle('pickle/mov_entities.pickle')
    entity_size        = len(entities)

# vocab = set(entities + ['directed_by', 'written_by', 'starred_actors', 'release_year', 'has_genre', 'has_tags', 'has_plot'] 
#                      + ['!directed_by', '!written_by', '!starred_actors', '!release_year', '!has_genre', '!has_tags', '!has_plot'] )
# for _, q, answer in train_data + test_data:
#     vocab |= set(q + answer)
# vocab = sorted(vocab)
# save_pickle(vocab, 'pickle/mov_vocab.pickle')
vocab = load_pickle('pickle/mov_vocab.pickle')
stopwords = load_pickle('pickle/mov_stopwords.pickle')

# w2i = dict((c, i) for i, c in enumerate(vocab))
# i2w = dict((i, c) for i, c in enumerate(vocab))
# save_pickle(w2i, 'mov_w2i.pickle')
# save_pickle(i2w, 'mov_i2w.pickle')
w2i = load_pickle('pickle/mov_w2i.pickle')
i2w = load_pickle('pickle/mov_i2w.pickle')

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) #+ 1
# story_maxlen = max(map(len, (x for x, _, _ in train_data + test_data)))
query_maxlen = max(map(len, (x for _, x, _ in train_data + test_data)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
# print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training data:', len(train_data))
print('Number of test data:', len(test_data))
print('-')
print('Here\'s what a "data" tuple looks like (input, query, answer):')
print(train_data[0])

load pickle/mov_task1_qa_pipe_train.pickle
load pickle/mov_task1_qa_pipe_test.pickle
load pickle/mov_kv_pairs.pickle
load pickle/mov_train_k.pickle
load pickle/mov_train_v.pickle
load pickle/mov_test_k.pickle
load pickle/mov_test_v.pickle
load pickle/mov_entities.pickle
load pickle/mov_vocab.pickle
load pickle/mov_stopwords.pickle
load pickle/mov_w2i.pickle
load pickle/mov_i2w.pickle
-
Vocab size: 74609 unique words
Query max length: 16 words
Number of training data: 96185
Number of test data: 9952
-
Here's what a "data" tuple looks like (input, query, answer):
([], ['what', 'movies', 'are', 'about', 'ginger rogers'], ['the barkleys of broadway', 'kitty foyle', 'top hat'])


In [5]:
train_k_max_memsize = max(map(len, (d for d in train_k)))
train_v_max_memsize = max(map(len, (d for d in train_v)))
test_k_max_memsize = max(map(len, (d for d in test_k)))
test_v_max_memsize = max(map(len, (d for d in test_v)))
print(train_k_max_memsize)
print(train_v_max_memsize)
print(test_k_max_memsize)
print(test_v_max_memsize)
print('--------')
# print(train_k_max_memsize, train_v_max_memsize)
print(train_data[22281])
len_list = []
for i, (k,v) in enumerate(zip(train_k, train_v)):
    if  len(k) == 0:
        print(i)
        print(train_data[i])
        print(k, v)
        break
#     len_list.append(len(k))
# print(sorted(len_list, reverse=True))

#     print(len(k), len(v))
#     if i > 500: break
# #     continue
#     if len(k) == 6041:
#         print(i)
#         pp.pprint(k[:10])
#         break

6101
6101
5909
5909
--------
([], ['the movie', 'a', 'boy', 'and', 'his', 'dog,', 'when', 'was', 'it', 'released'], ['1975'])
354
([], ['what', 'movies', 'can', 'be', 'described', 'by', 'writer'], ['waxworks', 'the lost weekend', 'sleuth', 'the raven', 'sinister', 'the ghost writer', 'almost famous', 'the shining'])
[] []


In [6]:
train_indices, test_indices = [], []
for i, k in enumerate(train_k):
    if len(k) != 0:
        train_indices.append(i)
for i, k in enumerate(test_k):
    if len(k) != 0:
        test_indices.append(i)
print(len(train_data), len(test_data))
train_data = [train_data[i] for i in train_indices]
train_k = [train_k[i] for i in train_indices]
train_v = [train_v[i] for i in train_indices]
test_data = [test_data[i] for i in test_indices]
test_k = [test_k[i] for i in test_indices]
test_v = [test_v[i] for i in test_indices]
print(len(train_data), len(test_data))

96185 9952
95521 9878


In [5]:
print(find_ngrams(entities, 'the movie a boy and his dog, when was it released'.split(' '), 100))
print(find_ngrams(entities, tokenize('the movie a boy and his dog, when was it released'), 100))
print(find_ngrams(entities, tokenize('w.s. van dyke was the director on which movies?'), 100))
print(find_ngrams(entities, word_tokenize('w.s. van dyke was the director on which movies?'), 100))

['the movie', 'a', 'boy', 'and', 'his', 'dog,', 'when', 'was', 'it', 'released']


NameError: name 'tokenize' is not defined

In [ ]:
# label_list = []
# for _,_,a in (train_data+test_data):
#     if a[0] not in label_list: label_list.append(a[0])
    
# w2i_label = dict((c, i) for i, c in enumerate(label_list))
# i2w_label = dict((i, c) for i, c in enumerate(label_list))

In [7]:
queries_train, answers_train = vectorize(train_data,
                                           w2i,
                                           query_maxlen)
queries_test, answers_test = vectorize(test_data,
                                            w2i,
                                            query_maxlen)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
# print('inputs_train shape:', inputs_train.shape)
# print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_test shape:', answers_test.shape)

-
inputs: integer tensor of shape (samples, max_length)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (95521, 16)
queries_test shape: (9878, 16)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_test shape: (9878, 74609)


In [8]:
from process_data import vectorize_kv
max_mem_len = 4
max_mem_size = 4
vec_train_k = vectorize_kv(train_k, max_mem_len, max_mem_size, w2i)
vec_train_v = vectorize_kv(train_v, max_mem_len, max_mem_size, w2i)
vec_test_k = vectorize_kv(test_k, max_mem_len, max_mem_size, w2i)
vec_test_v = vectorize_kv(test_v, max_mem_len, max_mem_size, w2i)

vectorize_kv: 0 / 95521
vectorize_kv: 5000 / 95521
vectorize_kv: 10000 / 95521
vectorize_kv: 15000 / 95521
vectorize_kv: 20000 / 95521
vectorize_kv: 25000 / 95521
vectorize_kv: 30000 / 95521
vectorize_kv: 35000 / 95521
vectorize_kv: 40000 / 95521
vectorize_kv: 45000 / 95521
vectorize_kv: 50000 / 95521
vectorize_kv: 55000 / 95521
vectorize_kv: 60000 / 95521
vectorize_kv: 65000 / 95521
vectorize_kv: 70000 / 95521
vectorize_kv: 75000 / 95521
vectorize_kv: 80000 / 95521
vectorize_kv: 85000 / 95521
vectorize_kv: 90000 / 95521
vectorize_kv: 95000 / 95521
vectorize_kv: 0 / 95521
vectorize_kv: 5000 / 95521
vectorize_kv: 10000 / 95521
vectorize_kv: 15000 / 95521
vectorize_kv: 20000 / 95521
vectorize_kv: 25000 / 95521
vectorize_kv: 30000 / 95521
vectorize_kv: 35000 / 95521
vectorize_kv: 40000 / 95521
vectorize_kv: 45000 / 95521
vectorize_kv: 50000 / 95521
vectorize_kv: 55000 / 95521
vectorize_kv: 60000 / 95521
vectorize_kv: 65000 / 95521
vectorize_kv: 70000 / 95521
vectorize_kv: 75000 / 95521
ve

In [9]:
chk_id = 100
print(train_data[chk_id][1])
print([w2i[w] for w in train_data[chk_id][1] if w in w2i])
# print([i2w[w] for w in train_data[0][1]])
print(queries_train[chk_id])
print([ i2w[i] for i in queries_train[chk_id]])
pp.pprint(train_k[chk_id])
pp.pprint(train_v[chk_id])
print(vec_train_k[chk_id])
print(answers_train[chk_id], i2w_label[np.argmax(answers_train[chk_id])])

['which', 'movies', 'can', 'be', 'described', 'by', 'robert schwentke']
[72580, 46780, 10202, 6723, 16929, 9914, 56132]
[72580 46780 10202  6723 16929  9914 56132     0     0     0     0     0
     0     0     0     0]
['which', 'movies', 'can', 'be', 'described', 'by', 'robert schwentke', '', '', '', '', '', '', '', '', '']
[   ['robert schwentke', '!directed_by'],
    ['robert schwentke', '!has_tags'],
    ['robert schwentke', '!directed_by'],
    ['robert schwentke', '!directed_by']]
[   ["the time traveler's wife"],
    ["the time traveler's wife"],
    ['flightplan'],
    ['r.i.p.d.']]
[[56132     1     0     0]
 [56132     4     0     0]
 [56132     1     0     0]
 [56132     1     0     0]]


NameError: name 'i2w_label' is not defined

In [ ]:
# 問題となる事例
1 the movie A Boy and His Dog, when was it released?	1975
1 Dance, Girl, Dance, when was it released?	1940
Dead Man Down written_by J.H. Wyman  # word_tokenize ['Dead', 'Man', 'Down', 'written_by', 'J.H', '.', 'Wyman']

load_task -> split(' 'JJ)
kv_pairs -> word_tokenizek

In [13]:
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Lambda, Permute, Dropout, add, multiply, dot

def MemNNKV(mem_len, mem_size, query_maxlen, vocab_size, embd_size, answer_size):
    print('mem_size:', mem_size)
    print('q_max', query_maxlen)
    print('embd_size', embd_size)
    print('vocab_size', vocab_size)
#     print('entity_size', entity_size)
    print('-----------')

    # placeholders
    key = Input((mem_size, mem_len,), name='Key_Input')
    val = Input((mem_size, mem_len,), name='Val_Input')
    question = Input((query_maxlen,), name='Question_Input')
    # print('key:', key.shape)

    shared_embd_A = Embedding(input_dim=vocab_size, output_dim=embd_size)

    key_encoded = shared_embd_A(key) # (None, mem_size, mem_len, embd_size)
    # print('key_encoded', key_encoded.shape)
    key_encoded = Lambda(lambda x: K.sum(x, axis=2)) (key_encoded) #(None, mem_size, embd_size)
    # print('key_encoded', key_encoded.shape)
    val_encoded = shared_embd_A(val) # (None, mem_size, embd_size)
    val_encoded = Lambda(lambda x: K.sum(x, axis=2)) (val_encoded)
    
    question_encoded = shared_embd_A(question) # (None, query_max_len, embd_size)
#     question_encoded = Lambda(lambda x: K.sum(x, axis=1)) (val_encoded)
    question_encoded = Lambda(lambda x: K.sum(x, axis=1)) (question_encoded) #(None, embd_size)
    # print('q_encoded', question_encoded.shape)
    q= question_encoded
    for h in range(2):
        # print('---hop', h)
        ph = dot([q, key_encoded], axes=(1, 2))  # (None, mem_size)
        # print('ph', ph.shape)
        ph = Activation('softmax')(ph)
        o = multiply([ph, Permute((2, 1))(val_encoded)]) # (None, embd_size, mem_size)
        # print('o', o.shape)
        o = Lambda(lambda x: K.sum(x, axis=2))(o) # (None, embd_size)
        # print('o', o.shape)
        R = Dense(embd_size, input_shape=(embd_size,), name='R_Dense_h' + str(h+1))
        q = R(add([q,  o])) # (None, embd_size)
        # print('q', q.shape)

#     answer = Dense(answer_size, name='last_Dense')(q) #(None, answer_size)
    answer = Dense(vocab_size, name='last_Dense')(q) #(None, vocab_size)
    # print('answer.shape', answer.shape)
    preds = Activation('softmax')(answer)
    
    # build the final model
    model = Model([key, val, question], preds)
    model.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy']) 
    return model


print('vec_test_k.shape', vec_train_k.shape)
print('vec_test_v.shape', vec_train_v.shape)
print('queries_train.shape', queries_train.shape)
print('ans', answers_train.shape)

vec_test_k.shape (95521, 4, 4)
vec_test_v.shape (95521, 4, 4)
queries_train.shape (95521, 16)
ans (95521, 74609)


In [14]:
embd_size = 500
memnn_kv = MemNNKV(max_mem_len, max_mem_size, query_maxlen, vocab_size, embd_size, None)
print(memnn_kv.summary())
now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
model_path = 'saved_models/' + now + '_kvnn-weights-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
memnn_kv.fit([vec_train_k, vec_train_v, queries_train], answers_train,
          batch_size=64,
          epochs=30,
          callbacks=callbacks_list,
          validation_data=([vec_test_k, vec_test_v, queries_test], answers_test))

mem_size: 4
q_max 16
embd_size 500
vocab_size 74609
-----------
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Question_Input (InputLayer)      (None, 16)            0                                            
____________________________________________________________________________________________________
Key_Input (InputLayer)           (None, 4, 4)          0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          multiple              37304500    Key_Input[0][0]                  
                                                                   Val_Input[0][0]                  
                                                                   Question_Input[0][0]             
___________________________

95521/95521 [==============================] - 75s - loss: 0.4086 - acc: 0.9121 - val_loss: 7.9578 - val_acc: 0.2393
Epoch 8/30
95521/95521 [==============================] - 75s - loss: 0.2463 - acc: 0.9502 - val_loss: 8.1375 - val_acc: 0.2381
Epoch 9/30
95521/95521 [==============================] - 75s - loss: 0.1670 - acc: 0.9690 - val_loss: 8.2872 - val_acc: 0.2376
Epoch 10/30
95521/95521 [==============================] - 75s - loss: 0.1253 - acc: 0.9778 - val_loss: 8.3987 - val_acc: 0.2395
Epoch 11/30
95521/95521 [==============================] - 75s - loss: 0.1019 - acc: 0.9819 - val_loss: 8.5249 - val_acc: 0.2352
Epoch 12/30
95521/95521 [==============================] - 75s - loss: 0.0874 - acc: 0.9850 - val_loss: 8.5823 - val_acc: 0.2377
Epoch 13/30
95521/95521 [==============================] - 75s - loss: 0.0788 - acc: 0.9860 - val_loss: 8.6457 - val_acc: 0.2390
Epoch 14/30
95521/95521 [==============================] - 75s - loss: 0.0712 - acc: 0.9876 - val_loss: 8.7096 

In [ ]:
memnn_kv.save('model_memnn_kv.h5')

## Predict

In [ ]:
model = load_model('demo_model_memnn_kv.h5') # heavy to run
# score = model.evaluate([vec_test_k, vec_test_v, queries_test], answers_test, verbose=1)
# score = model.evaluate([vec_train_k, vec_train_v, queries_train], answers_train, verbose=1)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

In [ ]:
# pred = model.predict([vec_train_k, vec_train_v, queries_train], verbose=1) 
pred = load_pickle('pred.pickle')

In [ ]:
for i, (p, a) in enumerate(zip(pred, answers_train[:len(pred)])):
    pw = i2w[np.argmax(p)]
    pa = i2w[np.argmax(a)]
    if pw == pa:
        print(i, ' '.join(train_data[i][1]))
        print(pw)


In [ ]:
print('q.shape', queries_test.shape)
ret_predict = model.predict([vec_test_k, vec_test_v, queries_test], verbose=1)

In [ ]:
print('q:',test_data[0][1])
print('predict:',i2w[np.argmax(ret_predict[0])])
print('label:',i2w[np.argmax(answers_test[0])])

## Interactive


In [ ]:
# stopwords = load_pickle('mov_stopwords.pickle')

# tokenize a question
q = 'who directed blade runner'
q_tokens = word_tokenize(q)
q_tokens = find_ngrams(vocab, q_tokens, 100000)
print('q_tokens:', q_tokens)

# vectorize a question
q_pad_len = max(0, query_maxlen - len(q_tokens))
vec_q = [w2i[w] for w in q_tokens] + [0] * q_pad_len
vec_q = np.array(vec_q)
vec_q = np.reshape(vec_q, (1, len(vec_q)))
print('vec_q:', vec_q)

# get related kv
k_list, v_list = [], []
for w in q_tokens:
    if w not in stopwords:
        for kv_ind, (k, v) in enumerate(kv_pairs):
            if w in (k+v):
                k_list += k
                v_list += v
    else:
        print(w, 'in stopwords')

def _vec_kv(data, w2i, mem_maxlen):
    vec = [w2i[e] for e in data if e in w2i]
    vec += [0] * max(0, mem_maxlen - len(vec))
    vec = vec[:mem_maxlen]
    vec = np.array(vec)
#     vec = np.expand_dims(vec, axis=0)
    print('---', vec.shape)
    vec = np.reshape(vec, (1, 100))
    
    return vec
# print(k_list)
# vectroize kv
vec_k, vec_v = None, None
vec_k = _vec_kv(k_list, w2i, mem_maxlen)
vec_v = _vec_kv(v_list, w2i, mem_maxlen)
# vec_k = [w2i[e] for e in k_list if e in w2i]
# vec_k += [0] * max(0, mem_maxlen - len(vec_k))
# vec_k = vec_k[:mem_maxlen]
# vec_k = np.array(vec_k)
# vec_v = [w2i[e] for e in v_list if e in w2i]
# vec_v += [0] * max(0, mem_maxlen - len(vec_v))
# vec_v = vec_v[:mem_maxlen]
# vec_v = np.array(vec_v)
print(vec_k.shape)

int_predict = model.predict([vec_k, vec_v, vec_q], batch_size=1, verbose=1)     
print('q:',q)
print('predict:',i2w[np.argmax(int_predict[0])])
# print('label:',i2w[np.argmax(answers_test[0])])

## misc